In [40]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.metrics import classification_report, confusion_matrix, f1_score

In [8]:
prefix = '../logs/'
os.listdir(prefix)

['results_svm_glove.csv',
 'results_svm.zip',
 'results_svm_ngram.csv',
 'results_rf.zip',
 'results_svm_laser.csv',
 'results_svm_tfidf.csv',
 'results_1649767718.009234.csv',
 'results.csv']

In [11]:
dfs = []
for fname in os.listdir('../logs/'):
    if 'results_svm_' in fname:
        df = pd.read_csv(prefix+fname)
        dfs.append(df)

In [33]:
final_df = pd.DataFrame()
for col_name in  df.columns:
    single_cols = pd.DataFrame(np.asarray([d[col_name] for d in dfs]).transpose())
    #print(single_cols)
    final_df[col_name] = single_cols.mode(axis=1)[0]
    

In [41]:
test_df = pd.read_csv('../data/development.csv')
def aggregate_users(df):
    columns_to_group_by_user = ['label', 'gender', 'profession', 'ideology_binary', 'ideology_multiclass']

    group = df.groupby(by = columns_to_group_by_user, dropna = False, observed = True, sort = False)

    # Custom df per user
    df_users = group[columns_to_group_by_user].agg(func = ['count'], as_index = False, observed = True).index.to_frame (index = False)

    merged_fields = []

    pbar = tqdm(df_users.iterrows(), total = df_users.shape[0], desc = "merging users")

    for index, row in pbar:
        df_user = df[(df['label'] == row['label'])]
        merged_fields.append({**row, **{field: ' [SEP] '.join (df_user[field].fillna ('')) for field in ['tweet']}})

    df = pd.DataFrame (merged_fields)
    return df


truth = aggregate_users(test_df)

merging users: 100%|██████████| 101/101 [00:00<00:00, 1331.39it/s]


In [45]:
f1s = []
for col_name in  df.columns[1:]:
    print(f"For label: {col_name}" )
    predictions = final_df[col_name]
    ytest = truth[col_name]
    cr = classification_report(ytest, predictions)
    score = f1_score(ytest, predictions , average='micro')
    f1s.append(score)
    print(cr)
    
    print(f'F1 : {score}')
    
print(f'Overall f1 score : {np.mean(f1s)}')

For label: gender
              precision    recall  f1-score   support

      female       0.87      0.94      0.90        48
        male       0.94      0.87      0.90        53

    accuracy                           0.90       101
   macro avg       0.90      0.90      0.90       101
weighted avg       0.90      0.90      0.90       101

F1 : 0.9009900990099011
For label: profession
              precision    recall  f1-score   support

  journalist       0.64      1.00      0.78        14
  politician       1.00      0.91      0.95        87

    accuracy                           0.92       101
   macro avg       0.82      0.95      0.86       101
weighted avg       0.95      0.92      0.93       101

F1 : 0.9207920792079208
For label: ideology_binary
              precision    recall  f1-score   support

        left       0.96      1.00      0.98        55
       right       1.00      0.96      0.98        46

    accuracy                           0.98       101
   macro avg 